<a href="https://colab.research.google.com/github/kghilothia06/emoji-predictor/blob/main/emoji_predictor_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Data Pre-Processing**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/emoji-predictor/Train.csv')
df_train.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
df_train.shape

(70000, 2)

In [6]:
df_train.head(10)

,TEXT,Label
0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,Been friends since 7th grade. Look at us now w...,2
3,This is what it looks like when someone loves ...,3
4,RT @user this white family was invited to a Bl...,3
5,"TRACK SEASON 2016, I'M READY FOR YA @ Universi...",4
6,Merry Christmas you filthy little animals. Wea...,5
7,"Westenders @user #LAZzNation @ Weston, Toronto\n",6
8,Maxwell heads home @ Summa Akron City Hospital\n,2
9,"Story On Saturday 136 Collins Ave, Miami Beach...",7


In [7]:
mapp = pd.read_csv('/content/drive/MyDrive/emoji-predictor/Mapping.csv')
mapp.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
mapp

,emoticons,number
0,😜,0
1,📸,1
2,😍,2
3,😂,3
4,😉,4
5,🎄,5
6,📷,6
7,🔥,7
8,😘,8
9,❤,9


In [9]:
df_test = pd.read_csv('/content/drive/MyDrive/emoji-predictor/Test.csv')
df_test.drop('Unnamed: 0',axis=1,inplace=True)

In [10]:
df_test.shape

(25958, 2)

In [11]:
df_test.head(10)

,id,TEXT
0,0,Thought this was cool...#Repost (get_repost)・・...
1,1,Happy 4th! Corte madera parade. #everytownusa ...
2,2,Luv. Or at least something close to it. @ Unio...
3,3,There's a slice of pie under that whipped crea...
4,5,#thankyou for your thank you We adore you both...
5,6,the SPECIAL4U Lyric video will be posted on my...
6,7,Momma Tanya's In town ! Awesome dinner @user w...
7,8,"Thing 1 and Thing 2 @ Huron, Ohio\n"
8,10,Bday girl and some random @ Sheraton New York ...
9,11,Always fun with my forever wedding date Congra...


In [12]:
X_train = df_train['TEXT']
y_train = df_train['Label']

X_test = df_test['TEXT']


In [13]:
#figure out max length of a sentence in X_train and X_test
maxlen=0
for i in X_train:
    maxlen = max(maxlen , len(i.split()))
    #print(len(i.split()))

for i in X_test:
    maxlen = max(maxlen , len(i.split()))    

print(maxlen)

34


In [14]:
X_train.shape[0]

70000

In [15]:
X_test.shape[0]

25958

In [16]:
#tokenization of a sentence
for ix in range(X_train.shape[0]):
    fd = X_train[ix].split()
    print(fd)
    for ij in range(len(fd)):
        print(fd[ij])
    break

['Vacation', 'wasted', '!', '#vacation2017', '#photobomb', '#tired', '#vacationwasted', '#mcgar30', '#miami', '@', 'Port', 'of…']
Vacation
wasted
!
#vacation2017
#photobomb
#tired
#vacationwasted
#mcgar30
#miami
@
Port
of…


# Converting Sentences into embeddings 
# using Glove vector

In [18]:
f = open('/content/drive/MyDrive/emoji-predictor/glove.6B.50d.txt' , encoding='utf8')

In [19]:
#make our own word embedding dictionary
embeddings_idx={}

for line in f:
    values = line.split()
    #print(values[0])
    #print(values[1:])
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    #print(word,coefs)
    embeddings_idx[word] = coefs

f.close()

In [20]:
embeddings_idx["miami"]

array([-0.59065 ,  0.48669 ,  0.23316 ,  1.0822  , -0.54907 , -0.96478 ,
       -1.7694  ,  0.42558 , -0.062791,  0.36638 , -0.50417 , -0.55302 ,
       -0.1254  ,  0.053595,  0.40962 , -0.35621 , -0.67128 , -0.47456 ,
       -1.0363  ,  0.090297, -0.74212 ,  0.49953 , -0.35469 ,  0.084452,
       -0.78938 , -1.3382  ,  0.33856 ,  0.012423,  0.51412 , -1.1261  ,
        1.6678  ,  0.15947 , -0.40807 , -1.1751  ,  0.4861  , -0.085435,
       -0.16804 , -0.14191 ,  0.88021 , -0.10223 , -0.6931  ,  0.62184 ,
        0.28272 ,  0.13934 , -0.63042 ,  0.79863 , -0.21831 , -0.41622 ,
       -0.15446 ,  0.58117 ])

function to make word embeddings for each word in each sentence

In [21]:
def embedding_output(X):
    #maxlen of a sentence
    maxlen = 50
    #embedding dimension
    emb_dim = 50
    
    batch_size = X.shape[0]
    #output of the function
    embedding_out = np.zeros((batch_size , maxlen , emb_dim))
    
    for ix in range(X.shape[0]):
        #tokenize current sentence
        X[ix] = X[ix].split()
        
        for ij in range(len(X[ix])):
            #get current word's embedding from 'glove' embeddings iff that word is present 
            if X[ix][ij] in embeddings_idx.keys():
                embedding_out[ix][ij] = embeddings_idx[X[ix][ij].lower()]
    
    return embedding_out
    

In [22]:
embedding_matrix_train = embedding_output(X_train)
embedding_matrix_test = embedding_output(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [23]:
X_train[0]

['Vacation',
 'wasted',
 '!',
 '#vacation2017',
 '#photobomb',
 '#tired',
 '#vacationwasted',
 '#mcgar30',
 '#miami',
 '@',
 'Port',
 'of…']

In [24]:
embedding_matrix_train.shape

(70000, 50, 50)

In [25]:
embedding_matrix_test.shape

(25958, 50, 50)